<a href="https://colab.research.google.com/github/SLVmain/NLP/blob/Les_09/HW_09_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 9. Языковое моделирование
Задание
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор

In [3]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [8]:
path1 = '/content/drive/MyDrive/GB_NLP_04_2022/09/evgenyi_onegin.txt'
path2 = '/content/drive/MyDrive/GB_NLP_04_2022/09/poems2.txt'

In [14]:
text1 = open(path1, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text1)))

Length of text: 286984 characters


In [12]:
print(text1[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [15]:
# сборник поэм
text2 = open(path2, 'rb').read().decode()
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text2)))

Length of text: 229064 characters


In [10]:
print(text2[:500])





Александр Пушкин

Поэмы





Кавказский пленник

Повесть






Посвящение

Н. Н. Раевскому[1 - Николай Николаевич Раевский – друг Пушкина в молодые годы.]


		     Прими с улыбкою, мой друг,
		     Свободной музы приношенье:
		Тебе я посвятил изгнанной лиры пенье
		     И вдохновенный свой досуг.
		Когда я погибал, безвинный, безотрадный,
		И шепот клеветы внимал со всех сторон,
		     Когда кинжал измены хладный,
		     Когда любви тяжелый сон
		     Меня терзали и мертвили,
		Я близ тебя е


In [16]:
text = text1 + text2

In [17]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

151 unique characters


In [18]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [19]:
text_as_int, text, len(text_as_int), len(text)

(array([ 85, 125, 119, ..., 135,  13,   1]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Святой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких дум и простоты;\n                        Но так и быть - рукой пристрастной\n                        Прими собранье пестрых глав,\n                        Полусмешных, полупечальных,\n                        Простонародных, идеальных,\n                        Небрежный плод моих забав,\n                        Бессонниц, легких вдохновений,\n                        Незрелых и увядших лет,\n                        Ума холодных наблюде

### train and target

In [20]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [21]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [22]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [23]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [ ]:
'''model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)'''

In [ ]:
'''class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)'''

In [59]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
'''def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model'''

In [60]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [61]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 151) # (batch_size, sequence_length, vocab_size)


In [62]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 128)           19328     
                                                                 
 lstm_8 (LSTM)               (64, None, 1024)          4722688   
                                                                 
 lstm_9 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_10 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_11 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 dense_2 (Dense)             (64, None, 151)           154775    
                                                                 
Total params: 30,074,903
Trainable params: 30,074,903


In [30]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 151), dtype=float32, numpy=
array([[ 2.2047789e-07, -4.8914817e-06, -5.5099990e-06, ...,
        -9.2169194e-06,  1.7902181e-05,  1.6183645e-05],
       [ 1.0852460e-05, -1.4476376e-05, -1.4937434e-05, ...,
        -1.5826337e-05,  6.7871639e-05,  6.5755878e-05],
       [ 3.7859561e-05, -3.0795494e-05, -3.1117801e-05, ...,
        -1.1158987e-05,  1.3899985e-04,  1.4028384e-04],
       ...,
       [-1.5309954e-03, -2.8812077e-03, -2.3487367e-04, ...,
         8.4412505e-04, -1.1169194e-03,  7.4520055e-04],
       [-1.3959256e-03, -2.6525301e-03, -1.7498263e-05, ...,
         7.5054588e-04, -1.0039614e-03,  8.4420957e-04],
       [-1.2657165e-03, -2.4156966e-03,  1.8689210e-04, ...,
         6.3297537e-04, -8.7093626e-04,  9.5175812e-04]], dtype=float32)>

In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [32]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'зана своим содержанием с той литературной борьбой, которую приходилось вести Пушкину в эти годы.\n\nК '

Next Char Predictions: 
 'mFgF[ёжШ*лхЖ)7ыЗzъGMEx\'«ЗЮ_aAя;g\n9и\tlk_нSЭN}жc-–ОЗМВЬyxВШ8еУCnЬ*vLиПЦЮ"’8пL\n;5эb"5лпGuTСхВhwК–Сд0Втю'


## Train the model

In [63]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 151)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.016933


In [64]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [37]:
#!rm -rf content/training_checkpoints

In [65]:
!ls /content/training_checkpoints

checkpoint		     ckpt_17.index
ckpt_11.data-00000-of-00001  ckpt_4.data-00000-of-00001
ckpt_11.index		     ckpt_4.index
ckpt_14.data-00000-of-00001  ckpt_7.data-00000-of-00001
ckpt_14.index		     ckpt_7.index
ckpt_17.data-00000-of-00001


In [66]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [67]:
EPOCHS = 300

In [68]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/300
79/79 [==============================] - 31s 336ms/step - loss: 2.8381
Epoch 2/300
79/79 [==============================] - 28s 348ms/step - loss: 2.2570
Epoch 3/300
79/79 [==============================] - 28s 346ms/step - loss: 1.9515
Epoch 4/300
79/79 [==============================] - 28s 353ms/step - loss: 1.8827
Epoch 5/300
79/79 [==============================] - 28s 345ms/step - loss: 1.8245
Epoch 6/300
79/79 [==============================] - 28s 345ms/step - loss: 1.7849
Epoch 7/300
79/79 [==============================] - 28s 353ms/step - loss: 1.7516
Epoch 8/300
79/79 [==============================] - 28s 345ms/step - loss: 1.7100
Epoch 9/300
79/79 [==============================] - 28s 345ms/step - loss: 1.6697
Epoch 10/300
79/79 [==============================] - 28s 345ms/step - loss: 1.6272
Epoch 11/300
79/79 [==============================] - 28s 353ms/step - loss: 1.5875
Epoch 12/300
79/79 [==============================] - 28s 345ms/step - loss: 1.5361
E

## Generate text

In [70]:
tf.train.latest_checkpoint(checkpoint_dir)

'/content/training_checkpoints/ckpt_298'

In [71]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [72]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 128)            19328     
                                                                 
 lstm_12 (LSTM)              (1, None, 1024)           4722688   
                                                                 
 lstm_13 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_14 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_15 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 151)            154775    
                                                                 
Total params: 30,074,903
Trainable params: 30,074,903


In [83]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

#через 20 эпох

In [54]:
#вывод через 20 эпох
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(len(text_))
text_

515


'И вот идет уже с дем упратный в нем не старать не удной взора на не постану наворить на спора одами и за кирей в вособо содерных в поэтой и то вах свители на поэтов в 18131 года, в поэте на слишет вострой котором света косторенный востиви (поэмы в топоро вмоля в строй соднанны када и с постера поэма послеваленный из на исти польбоводо                 Вот повреговал и не домой\n                       В ней отраный не востенечий.\n                          На пледы слебит как и леты,\n                              '

#через 300 эпох

In [80]:
#вывод через 300 эпох температура =1
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(len(text_))
print(text_)

515
И вот идет уже написаэь и в Тазечаете заря.],
		И страй-ко
                        Все благо: бдения и сна
                        Приходит час определенный;
         Убийцу волнуем симвора.

                                     XLI

                        Встает заря во мгле холодной
                    (песен и рассказа,
		Воспоминания полна,
		Быть может, повторит она
		Преданья грозного Кавказа;
		Рассеяны ль?


Мария

		               О милый мой,
		Ты будешь Угрюмый, однажоснулась роковая весть:
		Она з


In [78]:
#вывод через 300 эпох температура =0.5
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(len(text_))
print(text_)

515
И вот идет уже написанных строф,
                                               Потом свой кофе выпивал,
                        Плохой журнал перебирая,
                        Короте вы знаем, что Евгений
                        Издавна чтенье разлюбил,
                        Однако! уж ехать со двора.

                                     XXV

                        Он поскорей                                                                 Ноr.

                                                           


In [82]:
#вывод через 300 эпох температура =0.1
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(len(text_))
print(text_)

515
И вот идет уже написал
                                                                                         Ноr.

                                      X

                        Он пел любовь, любви послушный,
                                                                                                                                                                                                                                                                                                            


In [84]:
#вывод через 300 эпох температура =1
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(len(text_))
print(text_)

515
И вот идет уже наделала б вреда,
                        На этох глубоко и приворной
                        Назеровавы
		Раздался звучный глас Петра:
		«За дело, с Богом!» Из шатра,
		Толпой любимцев окруженный,
		Вымы Истрашные мечты.


Земфира

		Не верна поэтического сна
                        Иная, старая весна
                    Меня презреньем наконец
                        (будочник).



90

Лета – в я, церезкоздонь должны,
                        Так ваша верная подруга
            Так медленно по 
